In [46]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import scale

In [19]:
df = pd.read_csv('data/train/train.csv')
df

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14988,2,NaN,2,266,0,3,1,0,0,2,...,1,4,0,41326,61c84bd7bcb6fb31d2d480b1bcf9682e,0,I have 4 kittens that need to be adopt urgentl...,dc0935a84,3.0,2
14989,2,Serato & Eddie,60,265,264,3,1,4,7,2,...,1,2,0,41326,1d5096c4a5e159a3b750c5cfcf6ceabf,0,Serato(female cat- 3 color) is 4 years old and...,a01ab5b30,3.0,4
14990,2,Monkies,2,265,266,3,5,6,7,3,...,1,5,30,41326,6f40a7acfad5cc0bb3e44591ea446c05,0,"Mix breed, good temperament kittens. Love huma...",d981b6395,5.0,3
14991,2,Ms Daym,9,266,0,2,4,7,0,1,...,1,1,0,41336,c311c0c569245baa147d91fa4e351ae4,0,she is very shy..adventures and independent..s...,e4da1c9e4,3.0,4


In [20]:
#type : 1 is dog 2  is cat
# Breed :  use data/breed_labels.csv to get breed name
# Color :  use data/color_labels.csv to get color name
# State :  use data/state_labels.csv to get state name
# replace each type with its name
df['Type'] = df['Type'].replace([1,2],['Dog','Cat'])
# read breed labels and replace each breed id with its name
breed_labels = pd.read_csv('data/breed_labels.csv')
breed_labels = breed_labels.set_index('BreedID')
breed_labels = breed_labels.to_dict()['BreedName']
df['Breed1'] = df['Breed1'].replace(breed_labels)
df['Breed2'] = df['Breed2'].replace(breed_labels)
# read color labels and replace each color id with its name
color_labels = pd.read_csv('data/color_labels.csv')
color_labels = color_labels.set_index('ColorID')
color_labels = color_labels.to_dict()['ColorName']
df['Color1'] = df['Color1'].replace(color_labels)
df['Color2'] = df['Color2'].replace(color_labels)
df['Color3'] = df['Color3'].replace(color_labels)
# read state labels and replace each state id with its name
state_labels = pd.read_csv('data/state_labels.csv')
state_labels = state_labels.set_index('StateID')
state_labels = state_labels.to_dict()['StateName']
df['State'] = df['State'].replace(state_labels)
df

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,Cat,Nibble,3,Tabby,0,1,Black,White,0,1,...,1,1,100,Selangor,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,Cat,No Name Yet,1,Domestic Medium Hair,0,1,Black,Brown,0,2,...,1,1,0,Kuala Lumpur,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,Dog,Brisco,1,Mixed Breed,0,1,Brown,White,0,2,...,1,1,0,Selangor,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,Dog,Miko,4,Mixed Breed,0,2,Black,Brown,0,2,...,1,1,150,Kuala Lumpur,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,Dog,Hunter,1,Mixed Breed,0,1,Black,0,0,2,...,1,1,0,Selangor,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14988,Cat,NaN,2,Domestic Short Hair,0,3,Black,0,0,2,...,1,4,0,Selangor,61c84bd7bcb6fb31d2d480b1bcf9682e,0,I have 4 kittens that need to be adopt urgentl...,dc0935a84,3.0,2
14989,Cat,Serato & Eddie,60,Domestic Medium Hair,Domestic Long Hair,3,Black,Yellow,White,2,...,1,2,0,Selangor,1d5096c4a5e159a3b750c5cfcf6ceabf,0,Serato(female cat- 3 color) is 4 years old and...,a01ab5b30,3.0,4
14990,Cat,Monkies,2,Domestic Medium Hair,Domestic Short Hair,3,Cream,Gray,White,3,...,1,5,30,Selangor,6f40a7acfad5cc0bb3e44591ea446c05,0,"Mix breed, good temperament kittens. Love huma...",d981b6395,5.0,3
14991,Cat,Ms Daym,9,Domestic Short Hair,0,2,Yellow,White,0,1,...,1,1,0,Johor,c311c0c569245baa147d91fa4e351ae4,0,she is very shy..adventures and independent..s...,e4da1c9e4,3.0,4


In [21]:
# get the number of each type
df['Type'].value_counts()

Type
Dog    8132
Cat    6861
Name: count, dtype: int64

In [22]:
# get the number of each breed
df['Breed1'].value_counts()

Breed1
Mixed Breed                   5927
Domestic Short Hair           3634
Domestic Medium Hair          1258
Tabby                          342
Domestic Long Hair             296
                              ... 
Pointer                          1
Staffordshire Bull Terrier       1
Irish Wolfhound                  1
Irish Terrier                    1
Dutch Shepherd                   1
Name: count, Length: 176, dtype: int64

In [23]:
# get the number of each color
df['Color1'].value_counts()


Color1
Black     7427
Brown     3750
Golden     947
Cream      884
Gray       684
White      667
Yellow     634
Name: count, dtype: int64

In [24]:
# plot Breed1 vs average AdoptionSpeed sort by avg AdoptionSpeed
df.groupby('Breed1')['AdoptionSpeed'].mean().sort_values(ascending=False)

Breed1
Nebelung                 4.0
Australian Shepherd      4.0
Silver                   4.0
Spaniel                  4.0
Sheep Dog                4.0
                        ... 
Ragamuffin               1.0
Kuvasz                   1.0
Chow Chow                1.0
Sphynx (hairless cat)    0.0
Chinese Crested Dog      0.0
Name: AdoptionSpeed, Length: 176, dtype: float64

In [25]:
# round age to nearest 10 and plot it vs adoption speed 
df['Age'] = df['Age'].apply(lambda x: round(x, -1))
px.line(df.groupby(['Age', 'Type', 'Gender'])['AdoptionSpeed'].mean().reset_index().sort_values(by="Age"), x="Age", y="AdoptionSpeed", color='Gender', symbol='Type', title="Age vs AdoptionSpeed")

In [26]:
#one hot encode  type, breed, state, gender and color and concat them with the original data frame
ohe = pd.concat([df, pd.get_dummies(df['Type'], prefix='Type').astype(int).fillna(0)], axis=1)
ohe = pd.concat([ohe, pd.get_dummies(ohe['State'], prefix='State').astype(int).fillna(0)], axis=1)
# one hot encode Breed1 and Breed2 as one one hot encoding
# df = pd.concat([df, pd.get_dummies(df['Breed1'], prefix='Breed').astype(int)], axis=1)
# df = pd.concat([df, pd.get_dummies(df['Breed2'], prefix='Breed')], axis=1)
encoded_breed = (pd.get_dummies(df['Breed1'], prefix='Breed').astype(int) + pd.get_dummies(df['Breed2'], prefix='Breed').astype(int)).fillna(0)
ohe  = pd.concat([ohe, encoded_breed], axis=1)

encoded_color = (pd.get_dummies(ohe['Color1'], prefix='Color').astype(int) + pd.get_dummies(ohe['Color2'], prefix='Color').astype(int) + pd.get_dummies(ohe['Color3'], prefix='Color').astype(int)).fillna(0)
ohe  = pd.concat([ohe, encoded_color], axis=1)
ohe.drop(['Type', 'Breed1', 'Breed2', 'Color1', 'Color2', 'Color3', 'State'], axis=1, inplace=True)
ohe.drop(['Name', 'RescuerID', 'Description', 'PetID'], axis=1, inplace=True)
ohe

,Age,Gender,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,...,Breed_Yellow Labrador Retriever,Breed_Yorkshire Terrier Yorkie,Color_0,Color_Black,Color_Brown,Color_Cream,Color_Golden,Color_Gray,Color_White,Color_Yellow
0,0,1,1,1,2,2,2,1,1,100,...,0,0,0.0,0.0,0.0,0,0,0,1,0
1,0,1,2,2,3,3,3,1,1,0,...,0,0,0.0,0.0,0.0,0,0,0,0,0
2,0,1,2,2,1,1,2,1,1,0,...,0,0,0.0,0.0,0.0,0,0,0,1,0
3,0,2,2,1,1,1,2,1,1,150,...,0,0,0.0,0.0,0.0,0,0,0,0,0
4,0,1,2,1,2,2,2,1,1,0,...,0,0,0.0,0.0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14988,0,3,2,2,2,2,2,1,4,0,...,0,0,0.0,0.0,0.0,0,0,0,0,0
14989,60,3,2,2,1,1,1,1,2,0,...,0,0,0.0,0.0,0.0,0,0,0,1,1
14990,0,3,3,2,2,1,3,1,5,30,...,0,0,0.0,0.0,0.0,1,0,1,1,0
14991,10,2,1,1,1,1,1,1,1,0,...,0,0,0.0,0.0,0.0,0,0,0,1,1


In [27]:
target = pd.get_dummies(ohe.pop('AdoptionSpeed'))


In [33]:
# use decision tree to classify the data
clf = DecisionTreeClassifier(random_state=0)
ohe_train = ohe.sample(frac=0.8, random_state=0)
ohe_test = ohe.drop(ohe_train.index)
target_train = target.loc[ohe_train.index]
target_test = target.drop(target_train.index)
clf.fit(ohe_train, target_train)
clf.score(ohe_test, target_test)

/home/maxime/anaconda3/envs/petfinder_multi_modal_classfication/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/maxime/anaconda3/envs/petfinder_multi_modal_classfication/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/maxime/anaconda3/envs/petfinder_multi_modal_classfication/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/maxime/anaconda3/envs/petfinder_multi_modal_classfication/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstanc

0.3051017005668556

In [35]:
t_points = TSNE(n_components=2).fit_transform(ohe)

/home/maxime/anaconda3/envs/petfinder_multi_modal_classfication/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/maxime/anaconda3/envs/petfinder_multi_modal_classfication/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/maxime/anaconda3/envs/petfinder_multi_modal_classfication/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.



In [40]:
#breed value count 
# replace breeds that aren't in the top 5 with other
breed1_value_counts = df['Breed1'].value_counts()
breed1_value_counts = breed1_value_counts[breed1_value_counts > 175]
breed1_value_counts = breed1_value_counts.index.tolist()
main_breed = df['Breed1'].apply(lambda x: x if x in breed1_value_counts else 'Other')



In [41]:
px.scatter(x=t_points[:,0], y=t_points[:,1], color=main_breed, title="TSNE")

In [57]:
# plot the data using PCA
pca = PCA(n_components=2)
pca.fit(scale(ohe))
pca_points = pca.transform(scale(ohe))
fig = px.scatter(x=pca_points[:,0], y=pca_points[:,1], color=df["Type"], title="PCA")
fig.show()

/home/maxime/anaconda3/envs/petfinder_multi_modal_classfication/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/maxime/anaconda3/envs/petfinder_multi_modal_classfication/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/maxime/anaconda3/envs/petfinder_multi_modal_classfication/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/maxime/anaconda3/envs/petfinder_multi_modal_classfication/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstanc

In [56]:
#calculate the importance of each feature for PCA 
pca = PCA()
pca.fit(scale(ohe))
pd.DataFrame(zip(pca.components_[0], ohe.columns)).sort_values(by=0, ascending=False)

/home/maxime/anaconda3/envs/petfinder_multi_modal_classfication/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/maxime/anaconda3/envs/petfinder_multi_modal_classfication/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/maxime/anaconda3/envs/petfinder_multi_modal_classfication/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.



,0,1
12,0.483919,Type_Cat
92,0.324294,Breed_Domestic Short Hair
221,0.207370,Color_Gray
91,0.171891,Breed_Domestic Medium Hair
222,0.141023,Color_White
...,...,...
131,-0.076955,Breed_Labrador Retriever
0,-0.095059,Age
2,-0.130167,MaturitySize
141,-0.383231,Breed_Mixed Breed
